# Esercizio 4

1. Sono dati due set di documenti (italiano vs. inglese; 20 docs x 10 classi, vs. 20 docs x 20 classi) con caratteristiche diverse. Sceglierne uno.
2. Scrivere un programma che utilizzi una rappresentazione basata sul feature vector model
3. Il programma deve costruire i profili (Rocchio) relativi alle classi cui appartengono i documenti, e
4. Deve classificare i nuovi documenti (utilizzando come metrica di distanza la cosine similarity o una delle metriche derivate) in una delle classi date.
5. Per ciascuna classe prendiamo il 90% dei documenti per costruire i profili, e il 10% per testare
6. Opzionale: partizionare in 10 split il dataset, ripetendo la procedura 10 volte ('training' su 90% e test su 10%), calcolando la media dell'accuratezza così ottenuta

Scelte progettuali
1. Nell'implementazione del metodo di Rocchio partire assegnando a β e γ i valori di 16 e 4, rispettivamente
2. Algoritmo per calcolare i documenti NPOS?


In [1]:
# pip install spacy
# python -m spacy download it_core_news_sm

import os
import re
from nltk.corpus import stopwords
import spacy
import math
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

## Pre-processing

Metodo utilizzato per eseguire il preprocessing delle frasi, in cui vengono effettuate le seguenti operazioni:
- Rimozione della punteggiatura
- Trasformazione delle lettere in lowercase
- Lemmatizzazione di tutte le parole
- Tokenizzazione della frase
- Rimozione delle stop words

In [2]:
stop_words = set(stopwords.words('italian')) #remove stop words
lemmatizer = spacy.load('it_core_news_sm')

def pre_processing(document):
    document = re.sub(r'[^\w\s]',' ',document) #remove punctuation
    document = document.lower()
    document = lemmatizer(document)
    document = [token.lemma_ for token in document]
    document = [w for w in document if not w in stop_words]
    return document

## Classe Documento

Classe utilizzata per immagazzinare le informazioni di un singolo documento, tra cui
- Titolo del documento
- Classe di appartenenza
- Testo del documento
- Term vector ricavato

In [3]:
class Documento:
    # ============================================
    # Constructor
    # ============================================
    def __init__(self, titolo, classe, testo):
        self.titolo = titolo
        self.classe = classe
        self.testo = testo
        self.term_vector = []
        self.is_near_positive = False

    def __str__(self):
        return "Titolo: " + self.titolo + "\nClasse: " + self.classe

## Classe ListaDocumenti

Questa classe contiene la lista di tutti i documenti visti, andando a salvare tra le sue proprietà tutte le classi dei vari documenti e il vocabolario complessivo usato nei documenti.

Il metodo principale è "calculate_term_vectors" e serve, come indica il nome, per calcolare il term vector di un documento. Per fare ciò, per ogni parola del documento viene moltiplicato il term frequency per l'inverse document frequency

In [4]:
class ListaDocumenti:
    # ============================================
    # Constructor
    # ============================================
    def __init__(self):
        self.documenti = []
        self.classi = set()
        self.vocabolario = set()

    def __str__(self):
        return "#documenti: " + str(len(self.documenti)) + ", #classi: " + str(len(self.classi)) + ", #vocabolario: " + str(len(self.vocabolario))

    # ============================================
    # Private Methods
    # ============================================    
    #inverse document frequency
    def _idf(self, word):
        return math.log10(len(self.documenti)/sum([1.0 for i in self.documenti if word in i.testo]))

    #term frequency
    def _tf(self, word, document_words):
        return document_words.count(word)/len(document_words)

    # ============================================
    # Public Methods
    # ============================================
    def add_documento(self, documento):
        self.documenti.append(documento)
        self.classi.add(documento.classe)
        self.vocabolario.update(documento.testo)

    def calculate_term_vectors(self):
        for documento in self.documenti:
            weights = {}
            for word in self.vocabolario:
                weights.update({word: self._tf(word, documento.testo) * self._idf(word)})
            documento.term_vector = weights
            #print('term vector', documento.term_vector)
        

## Classe Prototipo

Classe che salva la classe di appartenenza del prototipo e il suo profile vector

In [5]:
class Prototipo:
    # ============================================
    # Constructor
    # ============================================
    def __init__(self, classe, profile_vector):
        self.classe = classe
        self.profile_vector = profile_vector

    def __str__(self):
        return "Classe: " + self.classe + "\nProfile vector: " + str(self.profile_vector)

## Classe Rocchio

Questa classe serve per implementare il metodo di Rocchio. Innanzitutto vengono salvate tra le sue proprietà la lista di prototipi calcolati, il vocabolario usato nei documenti e le classi dei documenti.

Il metodo principale è il metodo "train" che permette di andare a generare i prototipi di ogni classe. Per ogni classe viene lanciato il metodo "_calcola_prototipo_rocchio" che andrà a calcolare il prototipo avvalendosi degli esempi positivi e dei near positive.

In [14]:
class Rocchio:
    # ============================================
    # Constructor
    # ============================================
    def __init__(self, vocabolario, classi):
        self.prototipi = []
        self.vocabolario = vocabolario
        self.classi = classi
        
    def __str__(self):
        return "Classe: " + self.classe + "\nProfile vector: " + str(self.profile_vector)
    
    # ============================================
    # Private Methods
    # ============================================

    def _add_prototipo(self, prototipo):
        self.prototipi.append(prototipo) 

    # dato un documento positivo, restituisce il documento negativo più vicino
    def _get_nearest_doc(self, positive_doc, documenti_training):
        max_similarity = 0
        nearest_doc = None
        for doc in documenti_training:
            if doc.classe != positive_doc.classe and not doc.is_near_positive:
                positive_doc_s = np.array(list(positive_doc.term_vector.values())).reshape(1, -1)
                doc_s = np.array(list(doc.term_vector.values())).reshape(1, -1)
                similarity = cosine_similarity(positive_doc_s, doc_s)[0][0]
                if similarity > max_similarity:
                    max_similarity = similarity
                    nearest_doc = doc
                    
        nearest_doc.is_near_positive = True
        return nearest_doc
    
    #metodo per calcolare i documenti near positive
    def _near_positive_documents(self, classe_pos, documenti_training):
        near_positive_documents = []
        for doc in documenti_training:
            if doc.classe == classe_pos:
                near_positive_documents.append(self._get_nearest_doc(doc, documenti_training))

        return near_positive_documents

    def _calcola_prototipo_rocchio(self, classe, documenti_training, beta, gamma):
        contributi_pos = {}
        contributi_near_pos = {}
        for word in self.vocabolario:
            contributi_pos.update({word: 0})
            contributi_near_pos.update({word: 0})

        near_pos = self._near_positive_documents(classe, documenti_training)

        # contributo dei positivi
        count_pos = 0
        for doc in documenti_training:
            if doc.classe == classe:
                for word in self.vocabolario:
                    contributi_pos[word] += doc.term_vector[word]
                count_pos += 1
        for word in self.vocabolario:
            contributi_pos[word] /= count_pos

        # contributo dei near positive
        count_near_pos = 0
        for n_pos in near_pos:
            for word in self.vocabolario:
                contributi_near_pos[word] -= n_pos.term_vector[word]
            count_near_pos += 1
            n_pos.is_near_positive = False

        for word in self.vocabolario:
            contributi_near_pos[word] /= count_near_pos

        # calcolo del prototipo
        prototipo = {}
        for word in self.vocabolario:
            prototipo[word] = beta * contributi_pos[word] - gamma * contributi_near_pos[word]
    
        return Prototipo(classe, prototipo)
    
    # ============================================
    # Public Methods
    # ============================================    
    def train(self, documenti_training, beta=4, gamma=16):
        self.prototipi = []
        for classe in self.classi:
            self._add_prototipo(self._calcola_prototipo_rocchio(classe, documenti_training, beta, gamma))
    
    def score(self, test_set):
        score = 0
        for doc in test_set:
            max_similarity = 0
            doc_s = np.array(list(doc.term_vector.values())).reshape(1, -1)

            for prototipo in self.prototipi:
                prototipo_s = np.array(list(prototipo.profile_vector.values())).reshape(1, -1)
                similarity = cosine_similarity(doc_s, prototipo_s)[0][0]

                if similarity > max_similarity:
                    max_similarity = similarity
                    classe = prototipo.classe

            if classe == doc.classe:
                score += 1
                
        return score/len(test_set)*100

## Estrazione dei documenti

Estrazione dei documenti 

In [7]:
'''
Crea gli oggetti Documento e li inserise nell'oggetto lista_documenti
'''
def extract_documents():
    path = "data\docs_200"

    lista_documenti = ListaDocumenti()

    for file_name in os.listdir(path):
        if os.path.isfile(os.path.join(path, file_name)):
            file = open("data/docs_200/" + file_name, "r", encoding="utf-8")
            classe = file_name.split("_")[0]
            text = pre_processing(file.read().replace("\n", " ").replace("\"", ""))
            text = text[:25]

            lista_documenti.add_documento(Documento(file_name, classe, text))

    return lista_documenti

## Applicazione della Cross-validation

Funzione che attua la k-fold validation, dove i k fold sono specificati dal parametro "cv". I documenti nel dataset vengono innanzitutto mischiati e infine vengono estratti i k fold

In [8]:
def cross_val_score(lista_documenti, cv=5, gamma=16, beta=4):
    scores = []
 
    modello_rocchio = Rocchio(lista_documenti.vocabolario, lista_documenti.classi)

    np.random.seed()

    #shuffle documents
    documents_permutato = np.random.permutation(lista_documenti.documenti)

    #divide x in cv parti
    data_folds = np.array_split(documents_permutato, cv)

    for i in range(cv):
        current_data_training_set = np.concatenate(data_folds[:i] + data_folds[i+1:], axis=0)
        current_data_test_set = data_folds[i]

        modello_rocchio.train(current_data_training_set, gamma, beta)
        score = modello_rocchio.score(current_data_test_set)
        print("Fold " + str(i) + ": " + str(score) + " %")

        scores.append(score)
    
    return scores

## Applicazione del fine tuning

Esso serve per trovare gamma e beta migliori

- Beta: Il parametro beta controlla l'effetto del termine medio della classe negativa sul calcolo del vettore rappresentativo della classe di riferimento. Un valore più grande di beta enfatizza maggiormente il termine medio della classe negativa, mentre un valore più piccolo lo enfatizza meno. La scelta di beta dipende dalla distribuzione dei dati e dalle preferenze specifiche dell'utente. In generale, valori compresi tra 0 e 1 sono comuni per beta.
\\
- Gamma: Il parametro gamma influenza il calcolo del vettore rappresentativo della classe negativa. Un valore più grande di gamma riduce l'effetto dei documenti della classe negativa, mentre un valore più piccolo lo aumenta. La scelta di gamma dipende dalla distribuzione dei dati e dalle preferenze specifiche dell'utente. Valori compresi tra 0 e 1 sono comuni per gamma.

In [9]:
def fine_tuning(lista_documenti, cv=10):
    best_gamma = 0
    best_beta = 0
    best_accuracy = 0
    for i in range(20):
        for j in range(20):
            print("Iterazione", str(i+1), str(j+1))
            accuracies = cross_val_score(lista_documenti, cv=cv, gamma=i+1, beta=j+1)
            print('\n')
            for accuracy in accuracies:
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_gamma = i+1
                    best_beta = j+1

    return best_gamma, best_beta, best_accuracy

## Main

In [10]:
lista_documenti = extract_documents()

In [12]:
lista_documenti.calculate_term_vectors()

In [15]:
cross_val_score(lista_documenti, cv=10, gamma=11, beta=1)

Fold 0: 50.0 %
Fold 1: 65.0 %
Fold 2: 55.00000000000001 %
Fold 3: 55.00000000000001 %
Fold 4: 35.0 %
Fold 5: 60.0 %


KeyboardInterrupt: 

In [ ]:
#accuracies = cross_val_score(lista_documenti, cv=5, gamma=16, beta=4)
print("Best gamma, beta, accuracy: ", fine_tuning(lista_documenti))

Fold 0: 45.0 %
Fold 1: 100.0 %
Fold 2: 97.5 %
Fold 3: 100.0 %
Fold 4: 97.5 %
Fold 9: 20.0 %


Iterazione 4 12
Fold 0: 30.0 %
Fold 1: 40.0 %
Fold 2: 15.0 %
Fold 3: 25.0 %
Fold 4: 25.0 %
Fold 5: 30.0 %
Fold 6: 35.0 %
Fold 7: 15.0 %
Fold 8: 20.0 %
Fold 9: 25.0 %


Iterazione 4 13
Fold 0: 25.0 %
Fold 1: 40.0 %
Fold 2: 40.0 %
Fold 3: 30.0 %
Fold 4: 35.0 %
Fold 5: 20.0 %
Fold 6: 25.0 %
Fold 7: 20.0 %
Fold 8: 10.0 %
Fold 9: 20.0 %


Iterazione 4 14
Fold 0: 20.0 %
Fold 1: 60.0 %
Fold 2: 20.0 %
Fold 3: 30.0 %
Fold 4: 20.0 %
Fold 5: 25.0 %
Fold 6: 35.0 %
Fold 7: 15.0 %
Fold 8: 15.0 %
Fold 9: 35.0 %


Iterazione 4 15
Fold 0: 30.0 %
Fold 1: 20.0 %
Fold 2: 40.0 %
Fold 3: 40.0 %
Fold 4: 40.0 %
Fold 5: 20.0 %
Fold 6: 20.0 %
Fold 7: 15.0 %
Fold 8: 25.0 %
Fold 9: 25.0 %


Iterazione 4 16
Fold 0: 10.0 %
Fold 1: 50.0 %
Fold 2: 20.0 %
Fold 3: 45.0 %
Fold 4: 35.0 %
Fold 5: 5.0 %
Fold 6: 25.0 %
Fold 7: 15.0 %
Fold 8: 35.0 %
Fold 9: 10.0 %


Iterazione 4 17
Fold 0: 10.0 %
Fold 1: 20.0 %
Fold 2: 30.0 %
Fold 3